This script processes the test_2_data.csv file. It first removes duplicate rows, then extracts useful data into a new table

In [1]:
import time
# import requests
import pandas as pd
from ast import literal_eval
import numpy as np
import csv

In [2]:
# load data
pd.set_option('display.max_columns', 500)
start = time.perf_counter()
print(f'{time.strftime("%H:%M:%S", time.localtime())}: Loading Data...')

# read in data with proper formatting
cl_data = pd.read_csv('../data_out/cl_data.csv', header=None, names=['gid_code', 'address', 'coverage_response', 'offers']).replace({np.nan: 'None'})
# convert json strings to dicts
cl_data["coverage_response"] = cl_data["coverage_response"].apply(literal_eval)
cl_data["offers"] = cl_data["offers"].apply(literal_eval)

print(f'{time.strftime("%H:%M:%S", time.gmtime(time.perf_counter() - start))}: Data loaded.')
cl_data

19:38:11: Loading Data...
00:00:55: Data loaded.


,gid_code,address,coverage_response,offers
0,530330112001,552 S CLOVERDALE ST SEATTLE WA 98108 USA,"{'status': 0, 'message': 'SUCCESS', 'addrValIn...","{'fixedWirelessQualified': False, 'offersList'..."
1,530330112001,545 S SULLIVAN ST SEATTLE WA 98108 USA,"{'status': 0, 'message': 'SUCCESS', 'addrValIn...","{'fixedWirelessQualified': False, 'offersList'..."
2,530330112001,552 S CLOVERDALE ST SEATTLE WA 98108 USA,"{'status': 0, 'message': 'SUCCESS', 'addrValIn...","{'fixedWirelessQualified': False, 'offersList'..."
3,530330112001,552 S CLOVERDALE ST SEATTLE WA 98108 USA,"{'status': 0, 'message': 'SUCCESS', 'addrValIn...","{'fixedWirelessQualified': False, 'offersList'..."
4,530330112001,714 S DONOVAN ST SEATTLE WA 98108 USA,"{'status': 1, 'message': 'Address_Validation_S...",None
...,...,...,...,...
22196,530330071022,315 1ST AVE W SEATTLE WA 98119 USA,"{'status': 0, 'message': 'GREEN - exact match'...","{'fixedWirelessQualified': False, 'groupId': N..."
22197,530330071022,315 1ST AVE W SEATTLE WA 98119 USA,"{'status': 0, 'message': 'GREEN - exact match'...","{'fixedWirelessQualified': False, 'groupId': N..."
22198,530330071022,520 3RD AVE W SEATTLE WA 98119 USA,"{'status': 0, 'message': 'GREEN - exact match'...","{'fixedWirelessQualified': False, 'groupId': N..."
22199,530330071022,117 W MERCER ST SEATTLE WA 98119 USA,"{'status': 0, 'message': 'GREEN - exact match'...","{'fixedWirelessQualified': False, 'groupId': N..."


In [3]:
# select all rows with duplicate addresses
duplicates = cl_data[cl_data.duplicated(subset='address', keep=False)].sort_values(by='address')
duplicates

,gid_code,address,coverage_response,offers
2280,530330061003,10 E ROANOKE ST SEATTLE WA 98102 USA,"{'status': 0, 'message': 'GREEN - exact match'...","{'fixedWirelessQualified': False, 'groupId': N..."
2287,530330061003,10 E ROANOKE ST SEATTLE WA 98102 USA,"{'status': 0, 'message': 'GREEN - exact match'...","{'fixedWirelessQualified': False, 'groupId': N..."
3970,530330014003,10021 7TH AVE NW SEATTLE WA 98177 USA,"{'status': 0, 'message': 'SUCCESS', 'addrValIn...","{'fixedWirelessQualified': False, 'offersList'..."
3973,530330014003,10021 7TH AVE NW SEATTLE WA 98177 USA,"{'status': 0, 'message': 'SUCCESS', 'addrValIn...","{'fixedWirelessQualified': False, 'offersList'..."
19900,530330120003,10032 42ND AVE SW SEATTLE WA 98146 USA,None,None
...,...,...,...,...
4916,530330266002,9841 26TH AVE SW SEATTLE WA 98106 USA,None,None
16748,530330265002,9865 11TH AVE SW SEATTLE WA 98106 USA,"{'status': 1, 'message': 'Address_Validation_S...",None
16739,530330265002,9865 11TH AVE SW SEATTLE WA 98106 USA,None,None
16677,530330265001,9954 3RD LN SW SEATTLE WA 98106 USA,None,None


In [4]:
indexes_to_save = []
# for each tuple of duplicates, we want to keep the one with:
# 1: a list of offers,
# 2: a success/green coverage response
# 3: a yellow coverage response
# 4: any status 0 coverage response
# 5: any status 1 coverage response
# 6: any coverage response
# 7: any row

u = duplicates['address'].unique()
for adr in u: # for each batch of duplicates
    rows = duplicates[duplicates['address'].values == adr]
    found_row = False
    restriction = 0
    while(not found_row): # for each restriction
        for i in rows.index: # check if any of the duplicates meet the restriction
            if restriction == 0 and duplicates.loc[i].offers and duplicates.loc[i].offers['offersList']: 
                indexes_to_save.append(i)
                found_row = True
                break
            elif restriction == 1 and duplicates.loc[i].coverage_response and \
                    (('green' in duplicates.loc[i].coverage_response['message'].lower()) or \
                     ('success' in duplicates.loc[i].coverage_response['message'].lower())): 
                indexes_to_save.append(i)
                found_row = True
                break
            elif restriction == 2 and duplicates.loc[i].coverage_response and \
                ('yellow' in duplicates.loc[i].coverage_response['message'].lower()):
                indexes_to_save.append(i)
                found_row = True
                break
            elif restriction == 3 and duplicates.loc[i].coverage_response and duplicates.loc[i].coverage_response['status'] == 0:
                indexes_to_save.append(i)
                found_row = True
                break
            elif restriction == 4 and duplicates.loc[i].coverage_response and duplicates.loc[i].coverage_response['status'] == 1:
                indexes_to_save.append(i)
                found_row = True
                break
            elif restriction == 5 and duplicates.loc[i].coverage_response:
                indexes_to_save.append(i)
                found_row = True
                break
            elif restriction == 6:
                indexes_to_save.append(i)
                found_row = True
                break
            else:
                pass
        restriction += 1

In [5]:
indexes_to_drop = duplicates.index.difference(indexes_to_save)   # select indexes that are unwanted duplicates
cl_no_dupes = cl_data.drop(index=indexes_to_drop, inplace=False) # drop those indexes
print(sum(cl_no_dupes.duplicated(subset='address', keep=False))) # confirm there are no more duplicates

0


In [89]:
print(cl_no_dupes.iloc[100].keys())
cl_no_dupes.iloc[100]['offers']['offersList']
# first row will probably be addresses
# cl_data.head(10)
# if row has an offer list, make a new row for each list
# cl_data.reset_index(drop=True, inplace=True)
offers = pd.DataFrame()
offers = cl_no_dupes[['address', 'coverage_response', 'offers']].copy().reset_index(drop=True).rename(columns={'coverage_response':'adr_info', 'offers':'offer_details'}) # add interested columns
expanded_adr_check = pd.json_normalize(offers['adr_info']) # normalize coverage data
offers = offers.drop('adr_info', axis=1).join(expanded_adr_check) # join normalized coverage data
offers['offer_details'] = offers['offer_details'].apply(lambda x: x['offersList'] if x else None) # extract offers list
offers = offers.explode('offer_details').reset_index(drop=True) # explode offers list
expanded_offers = pd.json_normalize(offers['offer_details']) # normalize offers data
offers = offers.drop('offer_details', axis=1).join(expanded_offers) # join normalized offer data
print(offers.columns)
offers

# save offers to csv
# offers.to_csv("../data_out/expanded_CL_offers.csv")

Index(['gid_code', 'address', 'coverage_response', 'offers'], dtype='object')
Index(['address', 'status', 'message', 'leadIndicator', 'leadIndicatorStatus',
       'addressId', 'unitNumber', 'geoSecUnitId', 'googleInfo', 'below940',
       'existingService', 'expectedCompDate', 'lnppiMainDecision',
       'addrValInfo.result', 'addrValInfo.billingSource',
       'addrValInfo.fullAddress', 'addrValInfo.addressId',
       'addrValInfo.mduInfo.mduList', 'addrValInfo.wireCenter',
       'addrValInfo.nearMatchAddress', 'addrValInfo.nearMatchList',
       'addrValInfo.exactMatchAddress', 'addrValInfo.companyOwnerId',
       'loopQualInfo.message', 'loopQualInfo.messageDetail',
       'biwfInfo.fiberQualified', 'biwfInfo.redirectUrl', 'loopQualInfo',
       'biwfInfo', 'addrValInfo.mduInfo', 'downloadSpeed', 'uploadSpeed',
       'downloadSpeedMbps', 'uploadSpeedMbps', 'downloadDisplaySpeed',
       'uploadDisplaySpeed', 'internetTypeSortOrder', 'internetType',
       'productType', 'priceTyp

In [90]:
# select relevant columns address, message, downloadSpeedMbps, uploadSpeedMbps, price, description
offers_view_1 = offers[['address', 'message', 'status', 'downloadSpeedMbps', 'uploadSpeedMbps', 'price', 'description']].copy()
offers_view_1.to_csv("../data_out/up_down_price_CL.csv")